In [ ]:
from pynq import Overlay
overlay = Overlay('MainDesign.bit')

In [ ]:
overlay?

In [ ]:
from pynq.lib.iic import AxiIIC
import time
from IPython.display import clear_output

In [ ]:
iic = AxiIIC(overlay.ip_dict["axi_iic_0"])

# ----------------------------------ADXL345 CODE--------------------------------------

## Pinout
Pin1 : SCL
Pin2 : SDA

In [ ]:
def ADXL_set_register(register_address,data): #Usage: (0x__,0x__)
    a0 = register_address
    sending_data = [a0]
    sending_data.append(data)
    iic.send(ADXL_address,sending_data,len(sending_data),0)
    
def ADXL_Receive_Data():
    rx_data = bytes(6)
    iic.send(ADXL_address,[0x32],1,1)
    iic.receive(ADXL_address,rx_data,6,0)
    x_byte = int.from_bytes(rx_data[:2],byteorder='little',signed=True)
    y_byte = int.from_bytes(rx_data[2:4],byteorder='little',signed=True)
    z_byte = int.from_bytes(rx_data[4:6],byteorder='little',signed=True)
    print("X: ",round(float(x_byte/256),2))
    print("Y: ",round(float(y_byte/256),2))
    print("Z: ",round(float(z_byte/256)-1,2))

In [ ]:
ADXL_Enable_Register = 0x2D
ADXL_address = 0x53

In [ ]:
ADXL_set_register(ADXL_Enable_Register,0x08)

## Main Get Coords Loop

In [ ]:
for i in range(1000):
    clear_output(wait=True)
    ADXL_Receive_Data()
    time.sleep(0.1)

print("Done, restart :D")

# ------------------------------------VL6180x Code--------------------------------

In [ ]:
#Variable declarations
VL_address = 0x29

__VL6180X_IDENTIFICATION_MODEL_ID               = 0x0000
__VL6180X_IDENTIFICATION_MODEL_REV_MAJOR        = 0x0001
__VL6180X_IDENTIFICATION_MODEL_REV_MINOR        = 0x0002
__VL6180X_IDENTIFICATION_MODULE_REV_MAJOR       = 0x0003
__VL6180X_IDENTIFICATION_MODULE_REV_MINOR       = 0x0004
__VL6180X_IDENTIFICATION_DATE                   = 0x0006    # 16bit value
__VL6180X_IDENTIFICATION_TIME                   = 0x0008    # 16bit value

__VL6180X_SYSTEM_MODE_GPIO0                     = 0x0010
__VL6180X_SYSTEM_MODE_GPIO1                     = 0x0011
__VL6180X_SYSTEM_HISTORY_CTRL                   = 0x0012
__VL6180X_SYSTEM_INTERRUPT_CONFIG_GPIO          = 0x0014
__VL6180X_SYSTEM_INTERRUPT_CLEAR                = 0x0015
__VL6180X_SYSTEM_FRESH_OUT_OF_RESET             = 0x0016
__VL6180X_SYSTEM_GROUPED_PARAMETER_HOLD         = 0x0017

__VL6180X_SYSRANGE_START                        = 0x0018
__VL6180X_SYSRANGE_THRESH_HIGH                  = 0x0019
__VL6180X_SYSRANGE_THRESH_LOW                   = 0x001A
__VL6180X_SYSRANGE_INTERMEASUREMENT_PERIOD      = 0x001B
__VL6180X_SYSRANGE_MAX_CONVERGENCE_TIME         = 0x001C
__VL6180X_SYSRANGE_CROSSTALK_COMPENSATION_RATE  = 0x001E
__VL6180X_SYSRANGE_CROSSTALK_VALID_HEIGHT       = 0x0021
__VL6180X_SYSRANGE_EARLY_CONVERGENCE_ESTIMATE   = 0x0022
__VL6180X_SYSRANGE_PART_TO_PART_RANGE_OFFSET    = 0x0024
__VL6180X_SYSRANGE_RANGE_IGNORE_VALID_HEIGHT    = 0x0025
__VL6180X_SYSRANGE_RANGE_IGNORE_THRESHOLD       = 0x0026
__VL6180X_SYSRANGE_MAX_AMBIENT_LEVEL_MULT       = 0x002C
__VL6180X_SYSRANGE_RANGE_CHECK_ENABLES          = 0x002D
__VL6180X_SYSRANGE_VHV_RECALIBRATE              = 0x002E
__VL6180X_SYSRANGE_VHV_REPEAT_RATE              = 0x0031

__VL6180X_SYSALS_START                          = 0x0038
__VL6180X_SYSALS_THRESH_HIGH                    = 0x003A
__VL6180X_SYSALS_THRESH_LOW                     = 0x003C
__VL6180X_SYSALS_INTERMEASUREMENT_PERIOD        = 0x003E
__VL6180X_SYSALS_ANALOGUE_GAIN                  = 0x003F
__VL6180X_SYSALS_INTEGRATION_PERIOD             = 0x0040

__VL6180X_RESULT_RANGE_STATUS                   = 0x004D
__VL6180X_RESULT_ALS_STATUS                     = 0x004E
__VL6180X_RESULT_INTERRUPT_STATUS_GPIO          = 0x004F
__VL6180X_RESULT_ALS_VAL                        = 0x0050
__VL6180X_RESULT_HISTORY_BUFFER                 = 0x0052
__VL6180X_RESULT_RANGE_VAL                      = 0x0062
__VL6180X_RESULT_RANGE_RAW                      = 0x0064
__VL6180X_RESULT_RANGE_RETURN_RATE              = 0x0066
__VL6180X_RESULT_RANGE_REFERENCE_RATE           = 0x0068
__VL6180X_RESULT_RANGE_RETURN_SIGNAL_COUNT      = 0x006C
__VL6180X_RESULT_RANGE_REFERENCE_SIGNAL_COUNT   = 0x0070
__VL6180X_RESULT_RANGE_RETURN_AMB_COUNT         = 0x0074
__VL6180X_RESULT_RANGE_REFERENCE_AMB_COUNT      = 0x0078
__VL6180X_RESULT_RANGE_RETURN_CONV_TIME         = 0x007C
__VL6180X_RESULT_RANGE_REFERENCE_CONV_TIME      = 0x0080

__VL6180X_READOUT_AVERAGING_SAMPLE_PERIOD       = 0x010A
__VL6180X_FIRMWARE_BOOTUP                       = 0x0119
__VL6180X_FIRMWARE_RESULT_SCALER                = 0x0120
__VL6180X_I2C_SLAVE_DEVICE_ADDRESS              = 0x0212
__VL6180X_INTERLEAVED_MODE_ENABLE               = 0x02A3

### Functions for VL6180X(from Baset)

In [ ]:
def VL6180x_set_register(register_address,data): #Usage: VL6180x_set_register(0x____,0x__)
    a1 = (register_address >> 8) & 0xFF
    a0 = register_address & 0xFF
    sending_data = [a1,a0]
    sending_data.append(data)
    iic.send(VL_address,sending_data,len(sending_data),0)
    
def VL6180x_get_register(register_address):      #Usage: VL6180x_get_register(0x____)
    rx_data = bytes(1)
    a1 = (register_address >> 8) & 0xFF
    a0 = register_address & 0xFF
    sending_data = [a1,a0]
    iic.send(VL_address,sending_data,len(sending_data),1)
    iic.receive(VL_address,rx_data,1,0)
    return rx_data
    
def VL6180x_Run_Required_Settings():
    VL6180x_set_register(0x0207, 0x01)
    time.sleep(0.1)
    VL6180x_set_register(0x0208, 0x01)
    time.sleep(0.1)
    VL6180x_set_register(0x0096, 0x00)
    time.sleep(0.1)
    VL6180x_set_register(0x0097, 0xfd)
    time.sleep(0.1)
    VL6180x_set_register(0x00e3, 0x00)
    time.sleep(0.1)
    VL6180x_set_register(0x00e4, 0x04)
    time.sleep(0.1)
    VL6180x_set_register(0x00e5, 0x02)
    time.sleep(0.1)
    VL6180x_set_register(0x00e6, 0x01)
    time.sleep(0.1)
    VL6180x_set_register(0x00e7, 0x03)
    time.sleep(0.1)
    VL6180x_set_register(0x00f5, 0x02)
    time.sleep(0.1)
    VL6180x_set_register(0x00d9, 0x05)
    time.sleep(0.1)
    VL6180x_set_register(0x00db, 0xce)
    time.sleep(0.1)
    VL6180x_set_register(0x00dc, 0x03)
    time.sleep(0.1)
    VL6180x_set_register(0x00dd, 0xf8)
    time.sleep(0.1)
    VL6180x_set_register(0x009f, 0x00)
    time.sleep(0.1)
    VL6180x_set_register(0x00a3, 0x3c)
    time.sleep(0.1)
    VL6180x_set_register(0x00b7, 0x00)
    time.sleep(0.1)
    VL6180x_set_register(0x00bb, 0x3c)
    time.sleep(0.1)
    VL6180x_set_register(0x00b2, 0x09)
    time.sleep(0.1)
    VL6180x_set_register(0x00ca, 0x09)
    time.sleep(0.1)
    VL6180x_set_register(0x0198, 0x01)
    time.sleep(0.1)
    VL6180x_set_register(0x01b0, 0x17)
    time.sleep(0.1)
    VL6180x_set_register(0x01ad, 0x00)
    time.sleep(0.1)
    VL6180x_set_register(0x00ff, 0x05)
    time.sleep(0.1)
    VL6180x_set_register(0x0100, 0x05)
    time.sleep(0.1)
    VL6180x_set_register(0x0199, 0x05)
    time.sleep(0.1)
    VL6180x_set_register(0x01a6, 0x1b)
    time.sleep(0.1)
    VL6180x_set_register(0x01ac, 0x3e)
    time.sleep(0.1)
    VL6180x_set_register(0x01a7, 0x1f)
    time.sleep(0.1)
    VL6180x_set_register(0x0030, 0x00)
    print("Default Settings Loaded")
    
def get_distance():
    VL6180x_set_register(__VL6180X_SYSRANGE_START, 0x01)
    time.sleep(0.010)
    distance = VL6180x_get_register(__VL6180X_RESULT_RANGE_VAL)
    receivedIntData = int.from_bytes(distance,byteorder='little',signed=False)
    return receivedIntData

In [ ]:
VL6180x_Run_Required_Settings()

### Main Get_Distance Loop

In [ ]:
for i in range(1000):
    clear_output(wait=True)
    Distance = get_distance()
    print("Range: ",Distance,"mm")
    time.sleep(0.05)
    
print("Done, restart :D")

# LPS22DF Pressure Sensor I2C

In [23]:
LPS22DF_address = 0x5C # With SDO low
LPS22DF_CtrlReg1 = 0x10
LPS22DF_PressureLSBData_Register= [0x28] # Automatically toggles from 0x28 to 0x2C

In [24]:
def LPS22DF_set_register(register_address,data): #Usage: (0x__,0x__)
    sending_data = [register_address]
    sending_data.append(data)
    iic.send(LPS22DF_address,sending_data,len(sending_data),0)
    
def LPS22DF_Receive_Data():
    rx_data = bytes(5)
    iic.send(LPS22DF_address,LPS22DF_PressureLSBData_Register,1,1)  # Setting register to 0x28 to read pressure and temperature data
    iic.receive(LPS22DF_address,rx_data,5,0)
    Pressure_Data = int.from_bytes(rx_data[:3],byteorder='little',signed=True)
    Temper_Data = int.from_bytes(rx_data[3:5],byteorder='little',signed=True)
    print(f"Pressure(hPa): {float(Pressure_Data/4096):.2f}")
    print(f"Temp(C): {float(Temper_Data/100):.2f}")


In [25]:
LPS22DF_set_register(LPS22DF_CtrlReg1,0x08) # setting to 1hz continuous mode1

### Main Loop for Pressure and Temp Data

In [26]:
for i in range(1000):
    clear_output(wait=True)    
    print(i)
    LPS22DF_Receive_Data()
    time.sleep(0.5)

23
Pressure(hPa): 984.98
Temp(C): 26.86


KeyboardInterrupt: 